# - ***Marketing Analysis Using `RFM (Recency, Frequency and Monetary)` Framework***
#### `RFM Analysis is a marketing technique used to quantitatively rank and group customers based on the recency, frequency and monetary total of their recent transactions to identify the best customers and perform targeted marketing campaigns`

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("New Retail.csv")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,UnitPrice,Year,Month,Day,Hour,WeekDay
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,1.73,2009,12,1,7,Tuesday
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1.78,2009,12,1,7,Tuesday
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,1.78,2009,12,1,7,Tuesday
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,22.86,2009,12,1,7,Tuesday
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,19.20,2009,12,1,7,Tuesday


- I assume that the analysis is done after the a week from the campaign.

In [3]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
max_date = df["InvoiceDate"].max() 

# Add 7 days to the maximum date
today_date = max_date + pd.DateOffset(days=7)
print(f"today's date is {max_date}\nthe day the analysis is done date is {today_date}")

today's date is 2010-12-09 20:01:00
the day the analysis is done date is 2010-12-16 20:01:00


In [4]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                    "Invoice": lambda num: num.nunique(),
                                    "Price": lambda price: price.sum()                       
                                    })
rfm.columns = ["Recency", "Freqency", "Monetary"]
rfm

,Recency,Freqency,Monetary
Customer ID,,,
12346,171,11,206.36
12347,9,2,162.95
12348,80,1,14.39
12349,49,2,373.44
12351,17,1,49.46
...,...,...,...
18283,24,6,477.27
18284,73,1,41.09
18285,302,1,47.65


In [5]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4211.0,97.036333,96.592957,7.00,24.000,58.00,143.00,380.00
Freqency,4211.0,4.303491,7.467080,1.00,1.000,2.00,5.00,184.00
Monetary,4211.0,264.893972,577.427689,0.29,53.285,125.44,291.78,18264.79


- Now I will define the RFM Scores. This score defines how bad or how good this customer is interacting with our store 
    - - Recency represents How far was this customer's last purchase.
    - - Frequency represents How often this customer purchases.
    - - Monetary represents How much thi customer spent on our store.

In [6]:
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequeny_score"] = pd.qcut(rfm['Freqency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm

,Recency,Freqency,Monetary,recency_score,frequeny_score,monetary_score
Customer ID,,,,,,
12346,171,11,206.36,2,5,4
12347,9,2,162.95,5,2,3
12348,80,1,14.39,2,1,1
12349,49,2,373.44,3,2,5
12351,17,1,49.46,5,1,2
...,...,...,...,...,...,...
18283,24,6,477.27,4,5,5
18284,73,1,41.09,3,2,1
18285,302,1,47.65,1,2,2


- The Score can be defined by the three factors all combined or two of them. for our case Money may be misleading if someone buys an Expensive product for only one time this will give him high score for  Monetary but he is actually not a good customer as he bought only one time and maybe he bought a long time ago  

In [7]:
rfm["RFM_SCORE"] = rfm['recency_score'].astype(str) + rfm['frequeny_score'].astype(str)
rfm.sample(10)

,Recency,Freqency,Monetary,recency_score,frequeny_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
14650,230,1,27.40,1,1,1,11
14226,83,1,73.74,2,1,2,21
15235,11,6,108.92,5,5,3,55
15968,30,1,189.09,4,2,4,42
14032,8,6,187.33,5,4,4,54
12746,182,1,59.10,1,1,2,11
16077,267,1,5.45,1,2,1,12
12867,39,6,731.63,4,4,5,44
16890,118,2,35.88,2,3,1,23


In [8]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': "Can't Loose",
    r'3[1-2]': "About to Sleep",
    r'33': "Need Attention",
    r'[3-4][4-5]': "Loyal Customer",
    r'41': "Promising",
    r'51': "New Customer",
    r'[4-5][2-3]': "Potintial Loyalist",
    r'5[4-5]': "VIP"
}

rfm["Segments"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)
rfm.head()

,Recency,Freqency,Monetary,recency_score,frequeny_score,monetary_score,RFM_SCORE,Segments
Customer ID,,,,,,,,
12346,171,11,206.36,2,5,4,25,Can't Loose
12347,9,2,162.95,5,2,3,52,Potintial Loyalist
12348,80,1,14.39,2,1,1,21,Hibernating
12349,49,2,373.44,3,2,5,32,About to Sleep
12351,17,1,49.46,5,1,2,51,New Customer


- ***What is the percentage of each Segment in our dataset?***

In [9]:
rfm["Segments"].value_counts(normalize=True).round(3) * 100

Segments
Hibernating           23.8
Loyal Customer        17.3
VIP                   15.3
At Risk               14.2
Potintial Loyalist    11.9
About to Sleep         7.7
Need Attention         4.7
Promising              2.1
Can't Loose            1.8
New Customer           1.1
Name: proportion, dtype: float64